In [1]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
import numpy as np
import socket
from copy import deepcopy
import random
import struct
import pickle
import math
import subprocess
import re

In [2]:
def server_receive(client_socket, length): #int数组版
    data = b''
    while len(data) < length:
        packet = client_socket.recv(length - len(data))
        if not packet:
            break
        data += packet
    # 将接收到的数据按每32字节一个长整型进行解析
    received_data = [int.from_bytes(data[i:i+32], byteorder='big') for i in range(0, len(data), 32)]
    return received_data

def server_send(client_socket, arr): #int数组版
    # packed_data = pickle.dumps(arr)
    packed_data = b''.join(x.to_bytes(32, byteorder='big') for x in arr)
    client_socket.sendall(packed_data)

def server_receive_one(client_socket, length): #int数组版
    data = client_socket.recv(length)
    a = int(data)
    return a

def server_send_one(client_socket, arr): #int数组版
    client_socket.sendall(str(arr).encode()) 

In [3]:
def public_input_level(level, file_path): # for compute theta and gama
# 不用清除上一次执行痕迹，如果level符合起效就是合理的，如果level不符合也不会起效。
# 保持xor_theta与xor_theta_gama格式统一(为了代码复用我太难了)
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines[0] = f"level = {level}\n"
    with open(file_path, 'w') as file:
        file.writelines(lines)

def public_input_theta(level, theta, beta, file_path): # for compute gama - fss_base
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines[0] = f"level = {level}\n"
    lines[2] = f"beta = {beta}\n"
    lines[3] = f"theta = {theta}\n"
    with open(file_path, 'w') as file:
        file.writelines(lines)

def public_input_beta(level, path, beta): # for trim tree - write only
    file_path = "./MP-SPDZ/Programs/Source/xor_theta_gama.mpc"
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines[3] = f"beta[{level}] = {beta}\n"
    with open(file_path, 'w') as file:
        file.writelines(lines)

def public_input_path(level, path, file_path): # for binary search - read only
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines[4] = f"path[{level}] = {path}\n"
    with open(file_path, 'w') as file:
        file.writelines(lines)

def mpc_xor_theta_tao_gama(Za_j, level, timer, t): # for fss_multi
    
    file_path = "./MP-SPDZ/Programs/Source/xor_theta_gama.mpc"
    public_input_level(level, file_path)

    # 定义要执行的 Bash 脚本命令
    commands = f'''
    cd ./MP-SPDZ
    ./compile.py -B 256 xor_theta_gama.mpc
    echo {Za_j[0]} > Player-Data/Input-P1-0
    echo {Za_j[1]} >> Player-Data/Input-P1-0
    ./yao-party.x -OF theta_gama -p 1 xor_theta_gama
    '''
    # timer.resume()
    # 执行命令并捕获输出
    process = subprocess.Popen(commands, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    stdout, stderr = process.communicate()
    # timer.pause()
    with open(f"time{t}.txt", "a") as f:
        f.write("stdout:\n")
        f.write(stdout)  # 如果stdout是字节串，需要解码成字符串再写入文件
        f.write("\n\nstderr:\n")
        f.write(stderr)  # 如果stderr是字节串，需要解码成字符串再写入文件
    
    file_path = './MP-SPDZ/theta_gama-P1-0'

    with open(file_path, 'r') as file:
        lines = file.readlines()
    theta = int(lines[0].strip())
    tao0 = int(lines[1].strip())
    tao1 = int(lines[2].strip())
    gama = int(lines[3].strip())
    
    return theta, tao0, tao1, gama

def mpc_xor_gama(Za_j, level, theta, beta, timer): # fss_base
    
    file_path = "./MP-SPDZ/Programs/Source/xor_gama.mpc"
    public_input_theta(level, theta, beta, file_path)
    
    # 定义要执行的 Bash 脚本命令
    commands = f'''
    cd ./MP-SPDZ
    ./compile.py xor_gama
    echo {Za_j[0]} > Player-Data/Input-P1-0
    echo {Za_j[1]} >> Player-Data/Input-P1-0
    ./yao-party.x -OF gama -p 1 xor_gama
    '''
    # timer.resume()
    # 执行命令并捕获输出
    process = subprocess.Popen(commands, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    stdout, stderr = process.communicate()
    # timer.pause()
    with open("time.txt", "a") as f:
        f.write("stdout:\n")
        f.write(stdout)  # 如果stdout是字节串，需要解码成字符串再写入文件
        f.write("\n\nstderr:\n")
        f.write(stderr)  # 如果stderr是字节串，需要解码成字符串再写入文件
    
    file_path = './MP-SPDZ/gama-P1-0'

    with open(file_path, 'r') as file:
        line1 = file.readline().strip()
    gama = int(line1)
    
    return gama

def mpc_xor_theta_tao(Za_j, level, timer): # fss_base
    
    file_path = "./MP-SPDZ/Programs/Source/xor_theta.mpc"
    public_input_level(level, file_path)
    
    # 定义要执行的 Bash 脚本命令
    commands = f'''
    cd ./MP-SPDZ
    ./compile.py xor_theta
    echo {Za_j[0]} > Player-Data/Input-P1-0
    echo {Za_j[1]} >> Player-Data/Input-P1-0
    ./yao-party.x -OF theta -p 1 xor_theta
    '''
    # timer.resume()
    # 执行命令并捕获输出
    process = subprocess.Popen(commands, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    stdout, stderr = process.communicate()
    # timer.pause()
    with open("time.txt", "a") as f:
        f.write("stdout:\n")
        f.write(stdout)  # 如果stdout是字节串，需要解码成字符串再写入文件
        f.write("\n\nstderr:\n")
        f.write(stderr)  # 如果stderr是字节串，需要解码成字符串再写入文件
    
    file_path = './MP-SPDZ/theta-P1-0'
    with open(file_path, 'r') as file:
        lines = file.readlines()
    theta = int(lines[0].strip())
    tao0 = int(lines[1].strip())
    tao1 = int(lines[2].strip())
    
    return theta, tao0, tao1

def mpc_compare(level, sum, Sa_0, timer, t): # fss_base

    # 定义要执行的 Bash 脚本命令
    commands = f'''
    cd ./MP-SPDZ
    ./compile.py -B 256 compare
    echo {sum} > Player-Data/Input-P1-0
    echo {Sa_0} >> Player-Data/Input-P1-0
    ./yao-party.x -OF compare -p 1 compare
    '''
    # timer.resume()
    # 执行命令并捕获输出
    process = subprocess.Popen(commands, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    stdout, stderr = process.communicate()
    # timer.pause()
    with open(f"time{t}.txt", "a") as f:
        f.write("stdout:\n")
        f.write(stdout)  # 如果stdout是字节串，需要解码成字符串再写入文件
        f.write("\n\nstderr:\n")
        f.write(stderr)  # 如果stderr是字节串，需要解码成字符串再写入文件
    
    file_path = './MP-SPDZ/compare-P1-0'

    with open(file_path, 'r') as file:
        lines = file.readlines()
    equal = int(lines[0].strip())
    larger = int(lines[1].strip())
    
    file_path = "./MP-SPDZ/Programs/Source/xor_theta_gama.mpc"
    public_input_path(level+1, larger, file_path)
    
    return equal

In [4]:
def fss_read(arr, Ta):
    temp = 0
    for x, y in zip(arr, Ta):
        if y:
            temp = temp ^ x
    return temp

def fss_write(Ta, Ya, arr):
    # value_new = foq[level].writeStashValue.pop()
    # index, ifdown = foq[level].writeStashIndex.pop()
    # path = compute_path(foq, index)
    # value_ori = foq_read(level, ifdown, path, read_copy, Ta, foq)
    # value_deta = value_new ^ value_ori ^ beta # random
    for t, y, w in zip(Ta, Ya, arr):#Ta[level], Ya[level], foq[level].up/down
        if t:
            w = y ^ w            
    return arr

In [5]:
def extract_fractions_elements(arr, level):
    n = len(arr)
    for j in range(1, level + 1):
        fractions = [k / 2**j for k in range(1, 2**j, 2)]
        elements = [arr[int(f * n)] for f in fractions]
    return elements

In [6]:
import time

class Timer:
    def __init__(self):
        self.start_time = None
        self.total_pause_time = 0
        self.paused = False

    def start(self):
        if not self.start_time:
            self.start_time = time.time()
        else:
            print("Timer is already running.")

    def pause(self):
        if self.start_time and not self.paused:
            self.pause_start_time = time.time()
            self.paused = True
        else:
            print("Timer is not running or already paused.")

    def resume(self):
        if self.paused:
            self.total_pause_time += time.time() - self.pause_start_time
            self.paused = False
        else:
            print("Timer is not paused.")

    def stop(self):
        if self.start_time:
            if self.paused:
                self.resume()
            total_time = time.time() - self.start_time - self.total_pause_time
            print("Total time:", total_time, "seconds")
            self.start_time = None
            self.total_pause_time = 0
        else:
            print("Timer is not running.")
        return total_time

In [7]:
#MPC
#exchage XOR shares of array A
#path a[M]
#value beta

def Lsb(x):
    return x & 1

def fss_base(Sa_0, key, M, option, t): # , path, deta # M = levelCount-1
    cipher = AES.new(key, AES.MODE_CTR, nonce=key[-8:])
    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('127.0.0.1', 12347))
    server_socket.listen(1)
    client_socket, _ = server_socket.accept()    
    '''
    mask = server_receive(client_socket, 1024)
    '''

    # MPC
    Ta_0 = server_receive_one(client_socket, 1024)

    # 建立两个初始化为0的二维数组，各M+1行且第i行有2**i个元素
    Sa = [[0] *(2**i) for i in range(M+1)]
    Sa_ = deepcopy(Sa)
    Ta = deepcopy(Sa) 
    Tb = deepcopy(Sa) 
    theta = [0] * (M+1)
    tao = [[0] * (2) for i in range(M+1)]
    Za = deepcopy(tao) 
    Zb = deepcopy(tao) 
    Ya = [0] * (2**M)
    # a = [0] + path[:M] 
    timer = Timer()
    
    Sa[0][0] = int.from_bytes(Sa_0, byteorder='big')
    Sa_[0][0] = int.from_bytes(Sa_0, byteorder='big')
    #Sa[0][0] = Sa_0
    Ta[0][0] = Ta_0
    #Ta[0][0] = int.from_bytes(Ta_0, byteorder='big')

    for j in range(1, M+1):   #range[1, M]
        for jx in range(0, 2**(j-1)):            
            plaintext = Sa_[j-1][jx].to_bytes((Sa_[j-1][jx].bit_length() + 7) // 8, byteorder='big')
            Sa[j][2*jx] = int.from_bytes(cipher.encrypt(plaintext), byteorder='big')
            Sa[j][2*jx+1] = int.from_bytes(cipher.encrypt(plaintext), byteorder='big')
            Za[j][0]= Za[j][0] ^ Sa[j][2*jx]
            Za[j][1]= Za[j][1] ^ Sa[j][2*jx+1]
        
        # Zb[j] = server_receive(client_socket, 4096)
        # server_send(client_socket, Za[j])

        #MPC
        # theta[j] = Za[j][1^a[j]] ^ Zb[j][1^a[j]]
        # tao[j][0] = Lsb(Za[j][0]) ^ Lsb(Zb[j][0]) ^ a[j] ^ 1
        # tao[j][1] = Lsb(Za[j][1]) ^ Lsb(Zb[j][1]) ^ a[j]
        
        theta[j], tao[j][0], tao[j][1] = mpc_xor_theta_tao(Za[j], j, timer, t)
        tao[j][0] = Lsb(tao[j][0])
        tao[j][1] = Lsb(tao[j][1])
        
        for jx in range(0, 2**j):
            temp = math.floor(jx/2)
            Sa_[j][jx] = Sa[j][jx]
            Ta[j][jx] = Lsb(Sa[j][jx])
            if Ta[j-1][temp]:
                Sa_[j][jx] ^= theta[j]
                Ta[j][jx] ^= tao[j][Lsb(jx)]

    #MPC
    # Ga = Za[M][a[M]] ^ Zb[M][a[M]] ^ theta[M] ^ beta
    
    Ga = mpc_xor_gama(Za[M], M, theta[M])
    
    for jx in range(0, 2**M):
        Ya[jx] = Sa_[M][jx]
        if Ta[M][jx]:
            Ya[jx] ^= Ga
            
    client_socket.close()
    server_socket.close()

    return Ta, Ya

def fss_multi(Sa_0, key, M, write_arr, read_arr, option, t): # , path, deta #beta有levelCount-1个元素
    cipher = AES.new(key, AES.MODE_CTR, nonce=key[-8:])
    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('127.0.0.1', 12347+t))
    server_socket.listen(1)
    client_socket, _ = server_socket.accept()    
    '''
    mask = server_receive(client_socket, 1024)
    '''
    # MPC，也可以不用，毕竟从协议必然推断出对方的T0
    Ta_0 = server_receive_one(client_socket, 1024)
    timer = Timer()
    timer.start()
    
    # 建立两个初始化为0的二维数组，各M+1行且第i行有2**i个元素
    Sa = [[0] *(2**i) for i in range(M+1)]
    Sa_ = deepcopy(Sa)
    Ta = deepcopy(Sa) 
    Tb = deepcopy(Sa) 
    theta = [0] * (M+1)
    tao = [[0] * (2) for i in range(M+1)]
    Za = deepcopy(tao) 
    Zb = deepcopy(tao) 
    Ya = deepcopy(Sa)
    Ga = [0]*(M+1)
    # a = [0] + path[:M] 
    # beta = [0] + deta[:M]
    equal = 0 # 找到就为1，也可以不使用该关键字
    
    Sa[0][0] = int.from_bytes(Sa_0, byteorder='big')
    Sa_[0][0] = int.from_bytes(Sa_0, byteorder='big')
    #Sa[0][0] = Sa_0
    Ta[0][0] = Ta_0
    #Ta[0][0] = int.from_bytes(Ta_0, byteorder='big')
 
    for j in range(1, M+1):   #range[1, M]

        for jx in range(0, 2**(j-1)):            
            plaintext = Sa_[j-1][jx].to_bytes((Sa_[j-1][jx].bit_length() + 7) // 8, byteorder='big')
            Sa[j][2*jx] = int.from_bytes(cipher.encrypt(plaintext), byteorder='big')
            Sa[j][2*jx+1] = int.from_bytes(cipher.encrypt(plaintext), byteorder='big')
            Za[j][0]= Za[j][0] ^ Sa[j][2*jx]
            Za[j][1]= Za[j][1] ^ Sa[j][2*jx+1]
        
        # Zb[j] = server_receive(client_socket, 4096)
        # server_send(client_socket, Za[j])
        
        # MPC
        # theta[j] = Za[j][1^a[j]] ^ Zb[j][1^a[j]]
        # Ga[j] = Za[j][a[j]] ^ Zb[j][a[j]] ^ theta[j] ^ beta[j]
        timer.pause()
        if option == 'binary_search':
            arr_j = extract_fractions_elements(read_arr, j)
            sum = fss_read(arr_j, Ta[j])
            equal = mpc_compare(j, sum, Sa[0][0], timer, t)
            
        theta[j], tao[j][0], tao[j][1], Ga[j] = mpc_xor_theta_tao_gama(Za[j], j, timer, t)
        tao[j][0] = Lsb(tao[j][0])
        tao[j][1] = Lsb(tao[j][1])
        
        timer.resume()
        for jx in range(0, 2**j):
            temp = math.floor(jx/2)
            Sa_[j][jx] = Sa[j][jx]
            Ta[j][jx] = Lsb(Sa[j][jx])
            if Ta[j-1][temp]:
                Sa_[j][jx] ^= theta[j]
                Ta[j][jx] ^= tao[j][Lsb(jx)]

        for jx in range(0, 2**j):
            Ya[j][jx] = Sa_[j][jx]
            if Ta[j][jx]:
                Ya[j][jx] ^= Ga[j]

        if option == 'trim tree':
            equal = mpc_compare(j, sum, Sa[0][0], timer)
            # 假设server0是执行查询的一方（没有client的情况）
            # public_input_beta(j+1, beta)
            arr = fss_write(Ta[j], Ya[j], write_arr)
    local_time = timer.stop()
    client_socket.close()
    server_socket.close()

    return Ta, Ya, local_time

In [8]:
arr_ori = [i for i in range(2**15)]
mask = [random.randint(0, 2**15) for i in range(2**15)]
arr_mask = []
for a,m in zip(arr_ori, mask):
    arr_mask.append(a^m)


In [9]:
def AES_key_exchange():
    import socket
    from cryptography.hazmat.primitives.asymmetric import dh
    from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, ParameterFormat
    from cryptography.hazmat.primitives import serialization
    from cryptography.hazmat.backends import default_backend
    from cryptography.hazmat.primitives.kdf.hkdf import HKDF
    from cryptography.hazmat.primitives import hashes

    # 生成DH参数
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

    # 序列化DH参数以便共享
    parameters_bytes = parameters.parameter_bytes(Encoding.PEM, ParameterFormat.PKCS3)

    # 生成私钥
    private_key = parameters.generate_private_key()

    # 生成公钥
    public_key = private_key.public_key()

    # 将公钥序列化以便交换
    public_key_bytes = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)

    # 创建Socket并监听
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen(1)

    print("Server listening on port 65432...")
    conn, addr = server_socket.accept()

    # 发送DH参数给客户端
    conn.sendall(parameters_bytes)

    # 发送公钥给客户端
    conn.sendall(public_key_bytes)

    # 接收客户端的公钥
    peer_public_key_bytes = conn.recv(4096)

    # 反序列化对方的公钥
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())

    # 生成共享密钥
    shared_key = private_key.exchange(peer_public_key)

    # 使用HKDF对共享密钥进行派生，得到对称密钥
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=16,  # 设置为16字节
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    print("Derived Symmetric Key (Server):", derived_key)

    # 关闭连接
    conn.close()
    server_socket.close()

    return derived_key

In [10]:
# key = AES_key_exchange()
key = b'7\xed\xa7v\x03\xec\xfe\x9d\xac\xa7\xef\x86=c\xbe\x1a' #存一下，DH密钥交换太耗时间了
Sa_0 = get_random_bytes(16)

In [11]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('127.0.0.1', 12345))
server_socket.listen(1)
client_socket, _ = server_socket.accept()
server_send(client_socket, mask)
client_socket.close()
server_socket.close()

In [12]:
Sa = int.from_bytes(Sa_0, byteorder='big')
arr_mine = []
for j in mask:
    arr_mine.append(Sa ^ j)

In [13]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('127.0.0.1', 12346))
server_socket.listen(1)
client_socket, _ = server_socket.accept()

arr_other = server_receive(client_socket, (2**15) * 32)
server_send(client_socket, arr_mine)

client_socket.close()
server_socket.close()

In [14]:
arr = []
for i, j in zip(arr_mine, arr_other):
    arr.append(i ^ j)

In [15]:
def create_shape_matched_list(original_list, placeholder=None):
    # 创建一个与给定多重嵌套列表形状完全一致的新列表。
    if not isinstance(original_list, list):
        return placeholder
    return [create_shape_matched_list(item, placeholder) for item in original_list]

In [16]:
M = 29
Ta, Ya, time_ = fss_multi(Sa_0, key, M, arr_mask, arr, 'binary_search', 0)
with open("local_time.txt", "a") as f:
    f.write("Total Time: {} seconds \n".format(time_))

import re

# 创建一个正则表达式模式，用于匹配时间值
time_pattern = re.compile(r"Time = (\d+\.\d+) seconds")

# 初始化总时间
total_time = 0.0

# 从输出中查找所有的时间值并相加
with open("time0.txt", "r") as f:
    for line in f:
        match = time_pattern.search(line)
        if match:
            time_str = match.group(1)
            total_time += float(time_str)

# 将总时间写入文件
with open("mpc_time.txt", "a") as f:
    f.write("Total Time: {} seconds \n".format(total_time))

In [ ]:
temp = create_shape_matched_list(Ya)
for i, list in enumerate(Ya):
    for j, value in enumerate(list):
        temp[i][j] = value^value
temp

[[0],
 [0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
import re

# 创建一个正则表达式模式，用于匹配时间值
time_pattern = re.compile(r"Global data sent = (\d+\.\d+) MB")

# 初始化总时间
total_time = 0.0

# 从输出中查找所有的时间值并相加
with open("time0.txt", "r") as f:
    for line in f:
        match = time_pattern.search(line)
        if match:
            time_str = match.group(1)
            total_time += float(time_str)

In [ ]:
client_socket.close()
server_socket.close()

In [ ]:
import subprocess

# 调用rm命令删除文件
subprocess.run(["rm", "time0.txt"])

CompletedProcess(args=['rm', 'time0.txt'], returncode=0)